In [1]:
import pandas as pd
import numpy as np

from keras.models import Model, load_model
from keras.layers import Input, LSTM, Dense, Reshape
from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras import optimizers
import tensorflow as tf

import matplotlib.pyplot as plt

2022-11-29 12:52:50.367776: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-29 12:52:50.900553: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-29 12:52:50.900579: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-29 12:52:52.493290: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [2]:
train_data=pd.read_csv('english_hindi.csv')

In [3]:
train_data.head()

,JANAMDIVAS,^जन्मदिवस$
0,RAKHA,^रक्खा$
1,MILIJULI,^मिलीजुली$
2,JAANCHON,^जांचों$
3,CHAMKATA,^चमकता$
4,KAYIYON,^कईयों$


In [4]:
t_data=train_data[:1000]

In [5]:
target_words =  t_data['^जन्मदिवस$'].tolist()
input_words =  t_data['JANAMDIVAS'].tolist()

In [6]:
print(input_words[0])
print(target_words[0])
print(f"Total number of input words {len(input_words)}")
print(f"Total number of target words {len(target_words)}")

RAKHA
^रक्खा$
Total number of input words 1000
Total number of target words 1000


In [7]:
def build_vocab(input_words, target_words):
  input_vocab = set()
  target_vocab = set()
  for input_word, target_word in zip(input_words, target_words):
      input_vocab.update(set(input_word))
      target_vocab.update(set(target_word))
  return input_vocab, target_vocab

In [8]:
input_vocab, target_vocab = build_vocab(input_words, target_words)

In [9]:
print(f"Total hindi characters {len(input_vocab)}")
print(f"Total english characters {len(target_vocab)}")

Total hindi characters 26
Total english characters 59


In [10]:
print(f"Total hindi characters {len(input_vocab)}")
print(f"Total english characters {len(target_vocab)}")



Total hindi characters 26
Total english characters 59


In [11]:
num_encoder_tokens = len(input_vocab)
num_decoder_tokens = len(target_vocab)

In [12]:
num_encoder_tokens

26

In [13]:
max_encoder_seq_length = max([len(txt) for txt in input_words])
max_decoder_seq_length = max([len(txt) for txt in target_words])

In [14]:
encoder_input_data = np.zeros((len(input_words), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros((len(input_words), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros( (len(input_words), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

In [15]:
input_token_index = dict([(char, i) for i, char in enumerate(input_vocab)])
target_token_index = dict([(char, i) for i, char in enumerate(target_vocab)])

In [16]:
latent_dim = 256  # Latent dimensionality of the encoding space.
EMBEDDING_SIZE = 100

In [17]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
print(encoder_inputs.shape)

# Define embedding layer
encoder_embedding = tf.keras.layers.Embedding(num_encoder_tokens, EMBEDDING_SIZE, input_length=max_encoder_seq_length)
encoder_inputs_em = encoder_embedding(encoder_inputs)
print(encoder_inputs_em.shape)

# since embedding layer will add one more dim, we need to flatten last 2 dims
encoder_reshape = Reshape((-1, encoder_inputs_em.shape[2] * encoder_inputs_em.shape[3]))
encoder_inputs_re = encoder_reshape(encoder_inputs_em)
print(encoder_inputs_re.shape)

# Note that LSTM takes only 3 dims, (samples, timesteps, features)
encoder = LSTM(latent_dim, return_state=True)

encoder_outputs, state_h, state_c = encoder(encoder_inputs_re)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

(None, None, 26)


2022-11-29 12:53:21.326934: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-29 12:53:21.326961: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-29 12:53:21.326983: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (sahitya-Inspiron-15-3567): /proc/driver/nvidia/version does not exist
2022-11-29 12:53:21.327211: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


(None, None, 26, 100)
(None, None, 2600)


In [18]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
print(decoder_inputs.shape)

# Define embeddling layer
decoder_embedding = tf.keras.layers.Embedding(num_decoder_tokens, latent_dim)
decoder_inputs_em = decoder_embedding(decoder_inputs)
print(decoder_inputs_em.shape)

# since embedding layer will add one more dim, we need to flatten last 2 dims
decoder_reshape = Reshape((-1, decoder_inputs_em.shape[2] * decoder_inputs_em.shape[3]))
decoder_inputs_re = decoder_reshape(decoder_inputs_em)
print(decoder_inputs_re.shape)

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder = LSTM(latent_dim, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder(decoder_inputs_re, initial_state=encoder_states)
print(decoder_outputs.shape)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
print(decoder_outputs.shape)

(None, None, 59)
(None, None, 59, 256)
(None, None, 15104)
(None, None, 256)
(None, None, 59)


In [19]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 26)]   0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None, 59)]   0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 26, 10  2600        ['input_1[0][0]']                
                                0)                                                                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 59, 25  15104       ['input_2[0][0]']            

In [ ]:
# Training
batch_size = 64  # Batch size for training.
epochs = 500  # Number of epochs to train for.
initial_learning_rate = 0.001

# Add callbacks:
# 'EarlyStopping' to stop training when the model is not enhancing anymore
# 'ModelCheckPoint' to always keep the model that has the best val_accuracy
model_save_filename = "model.h5"

model.compile(
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=initial_learning_rate), 
    loss='categorical_crossentropy', 
    metrics=['accuracy'])

earlystopping_cb = EarlyStopping(
    monitor="val_accuracy",
    mode="max",
    verbose=1,
    patience=20, 
    restore_best_weights=True)

mdlcheckpoint_cb = ModelCheckpoint(
    model_save_filename, 
    monitor="val_accuracy", 
    mode="max",
    verbose=1,
    save_best_only=True
)

decay = initial_learning_rate / epochs
def lr_time_based_decay(epoch, lr):
    return lr * 1 / (1 + decay * epoch)
lrscheduler_cb = tf.keras.callbacks.LearningRateScheduler(lr_time_based_decay)

history = model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size, epochs=epochs,  validation_split=0.2, 
          callbacks=[earlystopping_cb, mdlcheckpoint_cb, lrscheduler_cb])

Epoch 1/500
13/13 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 1: val_accuracy improved from -inf to 0.00000, saving model to model.h5
13/13 [==============================] - 38s 3s/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 2/500


In [ ]:

model = load_model('/home/sahitya/Desktop/Projects/Transliteration/.ipynb_checkpoints/model.h5')

In [ ]:
model.layers[-1]

In [70]:
encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[6].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]

encoder_model = Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_inputs_em = decoder_embedding(decoder_inputs)

# since embedding layer will add one more dim, we need to flatten last 2 dims
decoder_reshape = Reshape((-1, decoder_inputs_em.shape[2] * decoder_inputs_em.shape[3]))
decoder_inputs_re = decoder_reshape(decoder_inputs_em)
print(decoder_inputs_re.shape)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_lstm = model.layers[7] # lstm_2
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs_re, initial_state=decoder_states_inputs
)

decoder_states = [state_h_dec, state_c_dec]

decoder_dense = model.layers[-1]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

(None, None, 17920)


In [71]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [72]:
def predict_target(input_string):
  # convert for encoding 
  input_data = np.zeros((1, max_encoder_seq_length, num_encoder_tokens), dtype='float32')
  for t, char in enumerate(input_string):
    input_data[0, t, input_token_index[char]] = 1.

  # Encode the input as state vectors.
  states_value = encoder_model(input_data)

   # Generate empty target sequence of length 1.
  target_seq = np.zeros((1, 1, num_decoder_tokens), dtype='float32')
  # Populate the first character of target sequence with the start character.
  target_seq[0, 0, target_token_index['^']] = 1.0

  stop_condition = False
  decoded_sentence = ""

  while True:
    output_tokens, d_h, d_c = decoder_model.predict([target_seq] + states_value)
    # Sample a token
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = reverse_target_char_index[sampled_token_index]

    # Exit condition: either hit max length
    # or find stop character.
    if sampled_char == "$" or len(decoded_sentence) > max_decoder_seq_length:
      break
      
    decoded_sentence += sampled_char

    # Update the target sequence (of length 1).
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, sampled_token_index] = 1.0

    # Update states
    states_value = [d_h, d_c]
  return decoded_sentence

In [74]:
test_input = 'ADI'
predicted_string = predict_target(test_input)
predicted_string

1/1 [==============================] - 0s 38ms/step


'ूऑऑैैैैैैैैठऽऽठठठठठॊऽॊ'